In [1]:
#get pos
def getPoint(line):
	pos = line.split(' ')
	return [float(pos[i]) for i in range(58)]

#square(dist)
def dist2(u, v):
	d2 = 0
	for i in range(58):
		d2 = d2 + (u[i] - v[i]) ** 2
	return d2

#compute mean(pos)
def sumPos(u, v):
	return [u[i] + v[i] for i in range(58)]

def meanPos(sumP, count):
	return [sumP[i] / count for i in range(58)]

#classify points
class cluster(object):
	def __init__(self, c, dist2):
		self.c = c
		self.dist2 = dist2
		return

	def __call__(self, p):
		minD2 = float('inf')
		index = 10
		for i in range(10):
			d2 = self.dist2(self.c[i], p)
			if d2 < minD2:
				index = i
				minD2 = d2
		return (index, (p, minD2, 1))

	def update(self, IPDC):
		sumDist = 0
		for index, PDC in IPDC:
			pos, dist, count = PDC
			self.c[index] = meanPos(pos, count)
			sumDist = sumDist + dist
		return sumDist

#get info of clusters
def reCenter(u, v):
	uP, uD2, uCount = u
	vP, vD2, vCount = v
	return (sumPos(uP, vP), uD2 + vD2, uCount + vCount)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
845,application_1551670200070_0157,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [4]:
data = sc.textFile('/user/hz333/data/hw2/Q4/data.txt')
c = sc.textFile('/user/hz333/data/hw2/Q4/c1.txt') #if use c2.txt, change here

pos = data.map(getPoint)
c = c.map(getPoint)
c = c.collect()

classify = cluster(c, dist2)

dist = []
for i in range(20):
	#(classIndex, (pos, dist, 1))
	res = pos.map(classify)
	res = res.reduceByKey(reCenter)
	res = res.collect()
	dist.append(classify.update(res))

In [5]:
for i in range(20):
	print('%d: %.3f' %(i+1, dist[i]))

1: 623660345.306
2: 509862908.298
3: 485480681.872
4: 463997011.685
5: 460969266.573
6: 460537847.983
7: 460313099.654
8: 460003523.889
9: 459570539.318
10: 459021103.342
11: 458490656.192
12: 457944232.588
13: 457558005.199
14: 457290136.352
15: 457050555.060
16: 456892235.615
17: 456703630.737
18: 456404203.019
19: 456177800.542
20: 455986871.027